In [1]:
#import libraries
import xarray as xr
import netCDF4
import cftime
import pyproj
import numpy as np
from datetime import datetime

# Path to the NetCDF file
fileinput = '/data/IMFSE/PythonCourse/CDS/CERRA/cerra-2000.nc'

# Open the dataset
ds = xr.open_dataset(fileinput)

# Check if 'valid_time' has the 'units' attribute for conversion
time_var = ds["valid_time"]

if 'units' in time_var.attrs:
    units = time_var.attrs['units']  # Get units if they exist
    times = cftime.num2date(time_var.values, units=units, calendar='standard')
else:
    # If no units are found, assume the time is already in a usable format
    print("No 'units' attribute found in 'valid_time'. Using the values as is.")
    times = time_var.values

# Update the 'valid_time' variable in the dataset (if converted)
ds["valid_time"] = ("valid_time", times)

# Coordinates of the center of the burned area
burned_area_lat = 41.709377863541654
burned_area_lon = 1.892273844304144

# Check the range of lat/lon in the dataset
print(f"Latitude range in dataset: {ds.latitude.min().item()} to {ds.latitude.max().item()}")
print(f"Longitude range in dataset: {ds.longitude.min().item()} to {ds.longitude.max().item()}")

# Ensure the coordinates are within the dataset bounds
if not (ds.latitude.min() <= burned_area_lat <= ds.latitude.max()) or not (ds.longitude.min() <= burned_area_lon <= ds.longitude.max()):
    raise ValueError("The coordinates of the burned area are outside the bounds of the dataset")

# Find the nearest grid point to the burned area location
abs_diff_lat = np.abs(ds.latitude - burned_area_lat)
abs_diff_lon = np.abs(ds.longitude - burned_area_lon)

# Find the index of the nearest grid point
lat_idx = abs_diff_lat.argmin().item()
lon_idx = abs_diff_lon.argmin().item()

# Ensure lat_idx and lon_idx are within the valid range of indices
if lat_idx < 0 or lat_idx >= ds.latitude.size or lon_idx < 0 or lon_idx >= ds.longitude.size:
    raise IndexError(f"Index out of range: lat_idx={lat_idx}, lon_idx={lon_idx}")

# Extract the data at the nearest grid point
wdir10 = ds['wdir10'][:, lat_idx, lon_idx]
si10 = ds['si10'][:, lat_idx, lon_idx]
r2 = ds['r2'][:, lat_idx, lon_idx]
t2m = ds['t2m'][:, lat_idx, lon_idx]

# Print out the extracted variables for verification
print(f"10m Wind Direction (wdir10): {wdir10}")
print(f"10m Wind Speed (si10): {si10}")
print(f"Relative Humidity at 2m (r2): {r2}")
print(f"Temperature at 2m (t2m): {t2m}")

# Optionally, plot the time series for the extracted data
wdir10.plot(label='10m Wind Direction')
si10.plot(label='10m Wind Speed')
r2.plot(label='2m Relative Humidity')
t2m.plot(label='2m Temperature')
plt.legend()
plt.show()



No 'units' attribute found in 'valid_time'. Using the values as is.
Latitude range in dataset: 20.292280999999985 to 75.34678492314951
Longitude range in dataset: -58.10508276341616 to 74.10508163756856


IndexError: range object index out of range